In [1]:
"""
1. Search for an open opportunity at your possible potential clients
2. Search for relevant information from the opening that might act as a good starting point for your business with this company
3. Go through all of your company's projects and find relevant projects
4. Get important to share information about these projects
5. Write a compelling cold email about how your firm can be of great service to your potential client
6. Repeat for ALL potential clients
"""

"\n1. Search for an open opportunity at your possible potential clients\n2. Search for relevant information from the opening that might act as a good starting point for your business with this company\n3. Go through all of your company's projects and find relevant projects\n4. Get important to share information about these projects\n5. Write a compelling cold email about how your firm can be of great service to your potential client\n6. Repeat for ALL potential clients\n"

In [2]:
#webscrapping
!pipenv install langchain_community

Loading .env environment variables...
Installing langchain_community...
✔ Installation Succeeded
Installing dependencies from Pipfile.lock (f6673e)...
All dependencies are now up-to-date!
Upgrading langchain_community in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠙ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠴ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠼ Locking packages...
Installing dependencies from Pipfile.lock (36fc99)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (36fc99)...


In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/director-software-engineering/job/R-43982")
page_data = loader.load().pop().page_content

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None, 
    max_retries=2,
)
response = llm.invoke("who is the prime minister of uae, no preamble")
print(response.content)

Mohammed bin Rashid Al Maktoum is the Vice President and Prime Minister of the UAE, however, the actual day-to-day work is carried out by Mohammed bin Rashid Al Maktoum's deputy, Mohammed bin Abdullah Al Gergawi, who is the Minister of Cabinet Affairs.


In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    I will give you scraped text from the job posting. 
    Your job is to extract the job details & requirements in a JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'. 
    Only return valid JSON. No preamble, please.
    Here is the scraped text: {page_data}
    """
)

chain_extract = prompt_extract | llm 
response = chain_extract.invoke(input={'page_data':page_data})
print(type(response.content))
print(response.content)

<class 'str'>
{
  "role": "Director, Software Engineering",
  "experience": {
    "required": 15,
    "leadership": 5,
    "experience": "High performing software engineering teams",
    "fields": [
      "Agile software development methodology and DevOps (CICD)",
      "Deploying monitoring and tracing solutions using tools like Splunk, SignalFx and New Relic",
      "Modern software architectural principles and patterns (REST, domain-driven design, microservices, etc.)",
      "Front-end web application tech (javascript, CSS, html5, Vue, React/redux)",
      "Distributed cloud systems tech (node.js, Python, EC2, Kubernetes, Lambda, DynamoDB, Elasticsearch, or Google Cloud/Azure equivalents)",
      "3D computer graphics development",
      "Implementing and integrating AI, Machine Learning and related data solutions"
    ]
  },
  "skills": [
    "Excellent verbal and written communication skills, both technical and business-facing",
    "Strength in strategic thinking and translating

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
print(type(json_response))

<class 'dict'>


In [17]:
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        for row in csv_reader:
            # Separate technical skills (list) and project link (string)
            skills = tuple(row[:-1])  # Exclude the last element (project link)
            project_link = row[-1]  # Get the last element (project link)
            data.append((skills, project_link))  # Create a tuple with skills and link
    return data

# Example usage:
file_path = 'sample_portfolio.csv'  # Replace with your actual file path
data = read_csv_file(file_path)

for skills, project_link in data:
    print(skills, project_link)
    #print(project_link)

('Python', ' SQL', ' Pandas')  https://github.com/user/project1
('SQL', ' Python', ' Airflow')  https://github.com/user/project2
('PySpark', ' Spark SQL', ' Delta Lake')  https://github.com/user/project3
('Machine Learning', ' Deep Learning', ' TensorFlow')  https://github.com/user/project4
('Data Engineering', ' ETL', ' ELT')  https://github.com/user/project5
('Cloud Platforms (AWS', ' GCP', ' Azure)')  https://github.com/user/project6
('Data Warehousing', ' Data Modeling', ' DBT')  https://github.com/user/project7
('Data Visualization', ' Power BI', ' Tableau')  https://github.com/user/project8
('MLOps', ' MLflow', ' Kubeflow')  https://github.com/user/project9
('Natural Language Processing (NLP)', ' NLTK', ' spaCy')  https://github.com/user/project10
('Computer Vision', ' OpenCV', ' TensorFlow')  https://github.com/user/project11
('Time Series Analysis', ' Forecasting', ' Prophet')  https://github.com/user/project12
('Data Cleaning', ' Data Wrangling', ' Pandas')  https://github.com

In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio_links')

if not collection.count():
    for skills, project_url in data:
        collection.add(
            documents=str(skills),
            metadatas={'portfolio_url':project_url},
            ids=[str(uuid.uuid4())]
        )

In [19]:
json_response['skills']

['Excellent verbal and written communication skills, both technical and business-facing',
 'Strength in strategic thinking and translating strategy to measurable objectives',
 'Track record of inspiring others & positive change in solution design, technical delivery, and process improvement',
 'Passion for learning new technologies, patterns, and practices']

In [20]:
portfolio_urls = collection.query(query_texts=json_response['skills'], n_results=2)
portfolio_urls

{'ids': [['53ee9f3b-d18d-4820-92e1-bb87ec13f1ff',
   '3e1b5163-d958-4ff4-851c-a60b86a61369'],
  ['7572aee3-5128-45d6-ade0-d15df9a4c053',
   '86b5a3fb-7e82-4c31-997e-1d0192bd0bc3'],
  ['c039d752-edc6-4f80-82d2-d7c5099379d9',
   '3e1b5163-d958-4ff4-851c-a60b86a61369'],
  ['c039d752-edc6-4f80-82d2-d7c5099379d9',
   '801c0131-b2bf-49d1-a17f-a4d52a6afcab']],
 'embeddings': None,
 'documents': [["('Data Engineering', ' ETL', ' ELT')",
   "('Data Governance', ' Data Quality', ' Metadata Management')"],
  ["('Statistical Analysis', ' Hypothesis Testing')",
   "('Time Series Analysis', ' Forecasting', ' Prophet')"],
  ["('Feature Engineering', ' Scikit-learn')",
   "('Data Governance', ' Data Quality', ' Metadata Management')"],
  ["('Feature Engineering', ' Scikit-learn')",
   "('Machine Learning', ' Deep Learning', ' TensorFlow')"]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'portfolio_url': ' https://github.com/user/project5'},
   {'

In [21]:
job_description = json_response['description']

In [24]:
email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your Role: Your name is Utsav, You are an incredible business development officer who knows how to get clients. You work for X Consulting firm, your firm works with all sorts of IT clients and provide solutions in the domain of Data Science and AI. 
    X AI focuses on efficient tailored solutions for all clients keeping costs down. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    and remove if any This email aims to
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)

chain_email = email_prompt | llm
response = chain_email.invoke({'job_description':job_description, 'portfolio_urls':portfolio_urls})
print(response.content)

Subject: Unlocking Innovation in Product Design Platforms

Dear Hiring Manager at Nike,

I came across your job posting for a Lead Development role in Product Design platforms and experiences. As a Business Development Officer at X Consulting firm, I'm excited to introduce our expertise in Data Science and AI solutions that can help you innovate and deliver business value at scale.

At X Consulting, we've helped numerous clients like [Client 1](https://github.com/user/project5) and [Client 2](https://github.com/user/project18) develop efficient product design platforms. Our team has experience in managing third-party engagements and leading software delivery organizations to achieve business goals.

Our portfolio showcases our capabilities in:

- Data Engineering: We've successfully implemented ETL and ELT processes for clients like [Client 3](https://github.com/user/project15) and [Client 4](https://github.com/user/project12).
- Statistical Analysis: Our team has expertise in hypothes